# d) Setup Environmental Flow Requirements

Calculated Environmental Flow exceedance percentiles for each day of the historic flow series. These are used later to determine environmental flows.

## Procedure

From: **Mott MacDonald** - *Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D*

Based on the environmental flow bands, the methodology adopted for the energy modelling is as below:

- The flow on the 8th, 18th and 28th day of each month is identified. 
- The equivalent percentile of this flow based on flow exceedance curves is calculated for that day from the 92 years of data.
- This percentile is used to decide the environmental flow release for the succeeding ~10 day periods of 1st to 10th, 11th to 20th and 21st to end of the month.
- Environmental flow in this band and month is released in the respective channels.

In this notebook the exceedance percentiles for each day of the historic flow series are calculated. These will later be used to lookup the required EWR values for each channel depending on the Environmental Assurance Category selected for the channel. 

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| ngonye_flow_daily.csv        | Notebook: c_flow_data | Daily Flow series              |



## Outputs
| File                       | Description                                 |
|----------------------------|---------------------------------------------|
| ngonye_flow_daily_ewrx.csv  | Daily flow series for Ngonye with EWR Exceedance values  |


## Parameters

In [1]:
#Papermill parameters
#input_file='CC/ngonye_daily_WC1_1'#'ngonye_daily'
input_file=''

In [2]:
input_data='./input_data/'
output_data='./output_data/'

## Libraries

In [3]:
import numpy as np
import pandas as pd

## Load Daily Data

In [6]:
daily=pd.read_csv(output_data + input_file + '.csv')
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
#daily.head(5)

## Build FDCs
Build FDCs for each of the 'measurement days' - 8th, 18th, 28th of each month

In [8]:
cols=['9-28','10-8','10-18','10-28','11-8','11-18','11-28','12-8','12-18','12-28',
      '1-8','1-18','1-28','2-8','2-18','2-28','3-8','3-18','3-28','4-8','4-18',
      '4-28','5-8','5-18','5-28','6-8','6-18','6-28','7-8','7-18','7-28','8-8','8-18','8-28','9-8','9-18']
ewr_fdcs=pd.DataFrame(columns=cols,index=np.arange(0,1.01,0.01))

for column in ewr_fdcs:   
    (month,day)=column.split('-')
    ewr_fdcs[column]=np.percentile(daily[(daily['Month']==int(month)) & (daily['Day']==int(day))]['Flow'] ,(1-ewr_fdcs.index)*100,interpolation='linear')
    
#ewr_fdcs.tail(5)

## Functions

In [9]:
#Gives the measure date (proceeding 8th, 18th or 28th) for the given date
def measureDate(obsDate):
    if obsDate.day <= 10:
        if obsDate.month==1:
            mn=13
            yr=obsDate.year-1
        else:
            mn=obsDate.month
            yr=obsDate.year
        return pd.Timestamp(yr, mn-1, 28)
    elif obsDate.day <=20:
        return pd.Timestamp(obsDate.year, obsDate.month, 8)
    else:
        return pd.Timestamp(obsDate.year, obsDate.month, 18)

#If it is a measure date (8th, 18th or 28th) then returns the ID of an FDC for that date. Otherwise False.
def isMeasureDay(obsDate):
    if obsDate.day==8 or obsDate.day==18 or obsDate.day==28:
        return str(obsDate.month) + '-' + str(obsDate.day) 
    else:
        return False
    
#Given an FDC ID and flow looks up exceedance
def lookupExceedance(measureDay,flow):
    return ewr_fdcs[ewr_fdcs[measureDay]<=flow].index.min()

## Exceedance Lookup
Build a table of measurement days and lookup the exceedance value for these from the FDCs already built

In [10]:
daily['dDate']=daily.index
daily['MeasureDay']=daily.apply(lambda x: isMeasureDay(x['dDate']),axis=1)
ewr_measure_days=pd.DataFrame(daily.loc[daily['MeasureDay']!=False]['MeasureDay'])
ewr_measure_days['Flow']=daily['Flow']
ewr_measure_days['Exceedance']=ewr_measure_days.apply(lambda x: lookupExceedance(x['MeasureDay'],x['Flow']),axis=1)
#ewr_measure_days.tail(3)

## EWR Measure Dates
Mark the dates on the daily timeseries where the EWR will be re-measured (8th, 18th, 28th of each month) 

In [11]:
daily['dDate']=daily.index
daily['EWRMeasureDate']=daily.apply(lambda x: measureDate(x['dDate']),axis=1)

## Daily EWR Exceedance
Lookup the EWR exceedance values for each day of the daily series based on the measure date

In [13]:
daily['EWRRefExceedance']=daily.join(ewr_measure_days,on='EWRMeasureDate',lsuffix='_l')['Exceedance']

In [14]:
daily=daily.drop(['MeasureDay','EWRMeasureDate','dDate'],axis=1)
daily=daily.fillna(0.5)
#daily.tail(3)

## Save

In [15]:
daily.to_csv(output_data + input_file + '.csv')